In [24]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [49]:
url = "https://www.imdb.com/list/ls006405458/"

In [50]:
def all_page_link(start_url):
    all_urls = []
    url = start_url
    while(url != None):            #Loop around all the required webpages and terminates when last page arive!
        all_urls.append(url)
        soup = BeautifulSoup(requests.get(url).text,"html.parser")
        next_links = soup.find_all(class_='flat-button lister-page-next next-page')    #Extracts the next page link.
        if (len(next_links) == 0):         # If their is no next page, it returns 0.
            url = None
        else:
            next_page = "https://www.imdb.com" + next_links[0].get('href')
            url = next_page
    return all_urls

In [51]:
def director_and_actor(Director_and_star):
    Director_and_star =  Director_and_star.replace("\n","")
    Director_and_star = Director_and_star.replace("|","")
    Director_and_star = Director_and_star.split("Stars:")
    Director_and_star[0] = Director_and_star[0].replace("Director:","")
    Director_and_star[0] = Director_and_star[0].replace("Directors:","")
    for i in range(10):
        Director_and_star[0]=Director_and_star[0].replace("  "," ")
    director = Director_and_star[0][1:]
    stars = Director_and_star[1]
    stars = stars.replace(":","")
    return director,stars

In [52]:
def votes_and_gross_conveter(votes_and_gross):
    votes_and_gross_list = []
    for i in votes_and_gross:
        votes_and_gross_list.append(i.text)
    if(len(votes_and_gross)==2):
        votes=votes_and_gross_list[0]
        gross = votes_and_gross_list[1]
    else:
        votes=votes_and_gross_list[0]
        gross = None
    
    return votes,gross

In [53]:
main_array = []
for url in tqdm(all_page_link(url)):     #Runs the function for all the pages.
    soup = BeautifulSoup(requests.get(url).text,"html.parser")         #Extracts out the main html code.
    for link in soup.find_all(class_='lister-item-content'):
        id = int(link.find('span',{"class":"lister-item-index unbold text-primary"}).text[:-1])
        name = link.find('a').text
        year = link.find('span',{"class":"lister-item-year text-muted unbold"}).text[1:5]
        run_time = link.find('span',{"class":"runtime"}).text
        genre = link.find('span',{"class":"genre"}).text[1:]
        rating = link.find('span',{"class":"ipl-rating-star__rating"}).text
        about = link.find_all('p')[1].text[5:]
        director,actors = director_and_actor(link.find_all('p',{"class":"text-muted text-small"})[1].text)
        votes, gross = votes_and_gross_conveter(link.find_all('span',{"name":"nv"}))
        votes = int(votes.replace(",",""))
        list_of_all = [id,name,year,run_time,genre,rating,about,director,actors,votes,gross]
        main_array.append(list_of_all)

  0%|                                                                                            | 0/1 [00:03<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'text'

In [54]:
index = ["id","name","year","run_time","genre","rating","about","director","actors","votes","gross"]

In [55]:
df = pd.DataFrame(main_array,columns=index)

In [56]:
df.head(5)

,id,name,year,run_time,genre,rating,about,director,actors,votes,gross
0,1,Climax!,1954,52 min,Drama,5.7,ican spy James Bond must outsmart card wiz and...,William H. Brown Jr.,"William Lundigan, Barry Nelson, Peter Lorre, L...",1184,None
1,2,Dr. No,1962,110 min,"Action, Adventure, Thriller",7.2,sourceful British government agent seeks answe...,Terence Young,"Sean Connery, Ursula Andress, Bernard Lee, Jos...",155242,$16.07M
2,3,From Russia with Love,1963,115 min,"Action, Adventure, Thriller",7.4,s Bond willingly falls into an assassination p...,Terence Young,"Sean Connery, Robert Shaw, Lotte Lenya, Daniel...",124843,$24.80M
3,4,Goldfinger,1964,110 min,"Action, Adventure, Thriller",7.7,"e investigating a gold magnate's smuggling, Ja...",Guy Hamilton,"Sean Connery, Gert Fröbe, Honor Blackman, Shir...",179231,$51.08M
4,5,Thunderball,1965,130 min,"Action, Adventure, Thriller",7,s Bond heads to the Bahamas to recover two nuc...,Terence Young,"Sean Connery, Claudine Auger, Adolfo Celi, Luc...",110763,$63.60M


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        27 non-null     int64 
 1   name      27 non-null     object
 2   year      27 non-null     object
 3   run_time  27 non-null     object
 4   genre     27 non-null     object
 5   rating    27 non-null     object
 6   about     27 non-null     object
 7   director  27 non-null     object
 8   actors    27 non-null     object
 9   votes     27 non-null     int64 
 10  gross     25 non-null     object
dtypes: int64(2), object(9)
memory usage: 2.4+ KB


In [58]:
df['about']

0     ican spy James Bond must outsmart card wiz and...
1     sourceful British government agent seeks answe...
2     s Bond willingly falls into an assassination p...
3     e investigating a gold magnate's smuggling, Ja...
4     s Bond heads to the Bahamas to recover two nuc...
5     et Agent James Bond and the Japanese Secret Se...
6     n early spy spoof, aging Sir James Bond comes ...
7     s Bond woos a mob boss' daughter and goes unde...
8     amond smuggling investigation leads James Bond...
9     s Bond is sent to stop a diabolically brillian...
10    s Bond is targeted by the world's most expensi...
11    s Bond investigates the hijacking of British a...
12    s Bond investigates the mid-air theft of a spa...
13    s Bond is assigned to find a missing British v...
14    ke Fabergé egg, and a fellow Agent's death, le...
15    P.E.C.T.R.E. Agent has stolen two American nuc...
16    recovery of a microchip off the body of a fell...
17    s Bond is sent to investigate a KGB policy

In [59]:
df.head()

,id,name,year,run_time,genre,rating,about,director,actors,votes,gross
0,1,Climax!,1954,52 min,Drama,5.7,ican spy James Bond must outsmart card wiz and...,William H. Brown Jr.,"William Lundigan, Barry Nelson, Peter Lorre, L...",1184,None
1,2,Dr. No,1962,110 min,"Action, Adventure, Thriller",7.2,sourceful British government agent seeks answe...,Terence Young,"Sean Connery, Ursula Andress, Bernard Lee, Jos...",155242,$16.07M
2,3,From Russia with Love,1963,115 min,"Action, Adventure, Thriller",7.4,s Bond willingly falls into an assassination p...,Terence Young,"Sean Connery, Robert Shaw, Lotte Lenya, Daniel...",124843,$24.80M
3,4,Goldfinger,1964,110 min,"Action, Adventure, Thriller",7.7,"e investigating a gold magnate's smuggling, Ja...",Guy Hamilton,"Sean Connery, Gert Fröbe, Honor Blackman, Shir...",179231,$51.08M
4,5,Thunderball,1965,130 min,"Action, Adventure, Thriller",7,s Bond heads to the Bahamas to recover two nuc...,Terence Young,"Sean Connery, Claudine Auger, Adolfo Celi, Luc...",110763,$63.60M


In [60]:
df.to_csv('fifth_27_jb.csv')